In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Plots
using Test
include("car.jl");
using ForwardDiff

  Activating environment at `c:\Users\ADMIN\Documents\GitHub\Project.toml`


LoadError: SystemError: opening file "c:\\Users\\ADMIN\\Documents\\GitHub\\ocrl_project\\car.jl": No such file or directory

In [ ]:
# Define the model
model = BicycleModel()

# get the number of states and controls
n = state_dim(model)
m = control_dim(model)

# Evaluate the continuous and discrete dynamics
x0 = SA[0.,0.,0.,0.]
u0 = SA[0.,0.]
t0 = 0.0
dt = 0.025
dynamics(model, x0, u0)
discrete_dynamics(RK4, model, x0, u0, t0, dt)  # use rk4 for integration

# Evaluate the continuous and discrete Jacobians
z0 = KnotPoint(x0,u0,dt,t0)   # create a `KnotPoint` type that stores everything together
∇f = RobotDynamics.DynamicsJacobian(model)
jacobian!(∇f, model, z0)
discrete_jacobian!(RK4, ∇f, model, z0)

# Extract pieces of the Jacobian
A = ∇f.A
B = ∇f.B;

In [ ]:
nx = n
nu = m

Q = Diagonal([1,1,1e-2,1e-2])
R = Diagonal([1e-1,1e-1])
Qf = Diagonal([1,1,1,1.])*10

N = 101 #401
Nmpc = 10 #60
Nhz = 1
delta = 12 #80
Nc = N + 2*delta

FD = ForwardDiff;

In [1]:
function get_trajectory(s)
    
    arr = split(s,"")
    if 60 < length(arr) < 120
        c = 2
    elseif length(arr) > 120
        c = 1
    else 
        c = 3
    end 

    N = length(arr) * c
   

    x1ref = zeros(N)*NaN
    x2ref = zeros(N)*NaN
    θref = zeros(N)*NaN
    v1ref = zeros(N)*NaN
    v2ref = zeros(N)*NaN
    ωref = zeros(N)*NaN
    rad = 20

    @show(size(θref))
    counter = 0
    for i in 1:length(arr)
        
        if arr[i] == "s"
            x1ref[i] += 0
            x2ref[i] += 0
            θref[i] += 0
        elseif arr[i] == "w"
            dir[i] = 1
        elseif arr[i] == "a"
            dir[i] = 2
        elseif arr[i] =="d"
            dir[i] = 3
        end
        
    end
    return dir,length(dir)
end

get_trajectory (generic function with 1 method)

In [ ]:
in = readline()
d1,N = get_trajectory(in)